In [4]:
import os

# docx2pdf is used for .doc/.docx on Windows/macOS (requires Word)
try:
    from docx2pdf import convert as convert_docx
except Exception:
    convert_docx = None

# Optional: pptxtopdf (user requested)
try:
    import pptxtopdf
except Exception:
    pptxtopdf = None

def convert_pptx_with_pptxtopdf(src, dst):
    """
    Convert PPTX to PDF using pptxtopdf.
    pptxtopdf.convert() treats the second argument as an output folder,
    so we pass the destination folder and then rename the generated file.
    """
    if pptxtopdf is None:
        raise RuntimeError("pptxtopdf not installed")
    
    # Extract destination folder and desired filename
    dst_folder = os.path.dirname(dst)
    dst_filename = os.path.basename(dst)
    base_name = os.path.splitext(dst_filename)[0]
    
    try:
        # pptxtopdf.convert(src, output_folder) writes to output_folder/<basename>.pdf
        pptxtopdf.convert(src, dst_folder)
        
        # Locate the generated PDF in the destination folder
        generated = os.path.join(dst_folder, base_name + ".pdf")
        if os.path.exists(generated):
            # Rename to exact dst if different
            if os.path.abspath(generated) != os.path.abspath(dst):
                os.replace(generated, dst)
            return
        
        # Fallback: search for any PDF with matching base name
        for f in os.listdir(dst_folder):
            if f.lower().startswith(base_name.lower()) and f.lower().endswith(".pdf"):
                generated = os.path.join(dst_folder, f)
                if os.path.abspath(generated) != os.path.abspath(dst):
                    os.replace(generated, dst)
                return
        
        raise RuntimeError(f"pptxtopdf did not produce expected PDF: {generated}")
    except Exception as e:
        raise RuntimeError(f"pptxtopdf conversion failed: {e}")

def convert_all_office_in_folder(folder_path, recursive=False, overwrite=False):
    """
    Convert .doc/.docx and .pptx files in folder_path to individual PDFs.
    - recursive: walk subfolders when True
    - overwrite: replace existing PDFs when True
    """
    if recursive:
        walker = ((root, files) for root, _, files in os.walk(folder_path))
    else:
        walker = ((folder_path, os.listdir(folder_path)),)

    for root, files in walker:
        for fname in files:
            ext = fname.lower().split('.')[-1]
            if ext not in ("doc", "docx", "pptx"):
                continue
            src = os.path.join(root, fname)
            dst = os.path.splitext(src)[0] + ".pdf"
            if os.path.exists(dst) and not overwrite:
                print(f"Skip (exists): {dst}")
                continue

            try:
                if ext in ("doc", "docx"):
                    if convert_docx is None:
                        print(f"Skipped (docx2pdf not available): {src}")
                        continue
                    convert_docx(src, dst)
                    print(f"Converted (docx2pdf): {src} -> {dst}")
                elif ext == "pptx":
                    if pptxtopdf is None:
                        print(f"Skipped (pptxtopdf not available): {src}")
                        continue
                    convert_pptx_with_pptxtopdf(src, dst)
                    print(f"Converted (pptxtopdf): {src} -> {dst}")
            except Exception as e:
                print(f"Failed: {src} ({e})")

if __name__ == "__main__":
    folder_path = r"Z:\OneDrive\Gardening_2025\LHH\All Documents"  # adjust as needed
    convert_all_office_in_folder(folder_path, recursive=False, overwrite=False)

Conversion completed: 1 files converted successfully, 0 files failed.
Converted (pptxtopdf): Z:\OneDrive\Gardening_2025\LHH\All Documents\Exploring a career change CYC EN Global Handout.pptx -> Z:\OneDrive\Gardening_2025\LHH\All Documents\Exploring a career change CYC EN Global Handout.pdf
Conversion completed: 1 files converted successfully, 0 files failed.
Converted (pptxtopdf): Z:\OneDrive\Gardening_2025\LHH\All Documents\Interviewing INT EN Global Handout.pptx -> Z:\OneDrive\Gardening_2025\LHH\All Documents\Interviewing INT EN Global Handout.pdf
Conversion completed: 1 files converted successfully, 0 files failed.
Converted (pptxtopdf): Z:\OneDrive\Gardening_2025\LHH\All Documents\Intro to ChatGPT GPT EN Global Handout.pptx -> Z:\OneDrive\Gardening_2025\LHH\All Documents\Intro to ChatGPT GPT EN Global Handout.pdf
Conversion completed: 1 files converted successfully, 0 files failed.
Converted (pptxtopdf): Z:\OneDrive\Gardening_2025\LHH\All Documents\Networking NET EN Global Handout.